In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/content/drive/Shareddrives/Shared_Stefano_Vicenzo

**CONTEXT**

For each timestamp in the context we have an array [A,B,C,D,(E that isn't present for now)]:

    "1597964400" : [ 50, 6, 0, null ]" 

* A: where the subject is
  * 100 = indoor
  * 0 = outdoor
  * 50 = no data
* B: staypoint id
  * if 0 is not at staypt
* C: path id
  * if 0 is not on path
* D: percentage of elevation
  * null if not applicable

**STAY PTS**

Useful and filtered info:

    "id" : "6",
    "types" : [ "HomeAndWork", "Building", "Residential" ],
    "indoor_prob" : 1.0,
    "greenspace_prob" : 0.0
  }

PS: startEndTimes non serve, ma mantenere per dopo. Duration varie sono inutili per ora. Usare per dopo

**PATH**

Useful and filtered info:

    "id" : "154",
    "freq" : 2,
    "freq_any_dir" : 2,
    "ele" : -1.394989,
    "dist" : 17.505,
    "types" : [ "" ],
    "confidence" : 0.0

PS: type possibile non serva a nulla se è sempre vuoto. Capire bene freq

In [2]:
!pip install matgrab

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install sktime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import scipy.io as spio
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import json
import math
import statistics
from itertools import cycle, zip_longest
import itertools
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

from sklearn.decomposition import PCA
import gc

gc.enable()

In [5]:
def loadmat(filename):
    '''
    this function should be called instead of direct spio.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    '''
    #print(filename)
    data = spio.loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_keys(data)

def _check_keys(dict):
    '''
    checks if entries in dictionary are mat-objects. If yes
    todict is called to change them to nested dictionaries
    '''
    for key in dict:
        if isinstance(dict[key], spio.matlab.mio5_params.mat_struct):
            dict[key] = _todict(dict[key])
    return dict        

def _todict(matobj):
    '''
    A recursive function which constructs from matobjects nested dictionaries
    '''
    dict = {}
    for strg in matobj._fieldnames:
        elem = matobj.__dict__[strg]
        if isinstance(elem, spio.matlab.mio5_params.mat_struct):
            dict[strg] = _todict(elem)
        else:
            dict[strg] = elem
    return dict

def print_mat_nested(d, indent=0, nkeys=0):  
    # Subset dictionary to limit keys to print.  Only works on first level
    if nkeys>0:
        d = {k: d[k] for k in list(d.keys())[:nkeys]}  # Dictionary comprehension: limit to first nkeys keys.

    if isinstance(d, dict):
        for key, value in d.items():         # iteritems loops through key, value pairs
          print('\t' * indent + 'Key: ' + str(key))
          print_mat_nested(value, indent+1)

    if isinstance(d,np.ndarray) and d.dtype.names is not None:  # Note: and short-circuits by default
        for n in d.dtype.names:    # This means it's a struct, it's bit of a kludge test.
            print('\t' * indent + 'Field: ' + str(n))
            print_mat_nested(d[n], indent+1)

In [6]:
def filter_values(context_file, df, num_ts):
  context = context_file['contextValues']
  start = 0
  end = 0
  filtered_ctx = []
  for i,t in enumerate(context.keys()):
    if int(t)>=math.trunc(df['Timestamp (ms)'].iloc[0]) and int(t)<=math.trunc(df['Timestamp (ms)'].iloc[-1]):
      filtered_ctx.append([int(t), context[t][0], context[t][1], context[t][2], context[t][3]])
  
  filtered_ctx = list(itertools.chain.from_iterable(itertools.repeat(x, 100) for x in filtered_ctx))[:num_ts]
  return filtered_ctx

In [7]:
def checkIndoor(filtered_ctx, staypts):
  indoor=[]
  for j,el in enumerate(filtered_ctx):
    #stay_id.append(filtered_ctx[j][2])
    if filtered_ctx[j][1]!=50:
      if filtered_ctx[j][1]>50:
        indoor.append(1)
      else:
        indoor.append(0)
    else:
      indoor.append(0.5)
  return indoor

In [8]:
def makeData(data_folder, data_format, ts=997417, verbose=False):
  #i=0
  full_df = pd.DataFrame()
  for folder in os.listdir(data_folder):
    for i,patient in enumerate(os.listdir(os.path.join(data_folder, folder))):
      #try:
        print(f"Processing patient id: {patient}")
        #-------------LOAD DATA-------------
        filemat = os.path.join(data_folder, folder, patient, 'Out of Lab', data_format )
        matdata = loadmat(filemat)  
        recording = matdata['data']['TimeMeasure1']['Recording4']
        if verbose:
          print_mat_nested(recording)

        #---------CREATE RAW DATASET--------
        df = pd.json_normalize(recording)

        df_triaxial = pd.DataFrame()
        for c in df.columns:
          if 'Acc' in c and 'Fs' not in c:
            df_triaxial[f'{c}_X (g)'] = pd.Series([v[0] for v in df[c].values[0]])
            df_triaxial[f'{c}_Y (g)'] = pd.Series([v[1] for v in df[c].values[0]])
            df_triaxial[f'{c}_Z (g)'] = pd.Series([v[2] for v in df[c].values[0]])
          elif 'Gyr' in c and 'Fs' not in c:
            df_triaxial[f'{c}_X (deg/s)'] = pd.Series([v[0] for v in df[c].values[0]])
            df_triaxial[f'{c}_Y (deg/s)'] = pd.Series([v[1] for v in df[c].values[0]])
            df_triaxial[f'{c}_Z (deg/s)'] = pd.Series([v[2] for v in df[c].values[0]])
          elif 'Mag' in c and 'Fs' not in c:
            df_triaxial[f'{c}_X (uT)'] = pd.Series([v[0] for v in df[c].values[0]])
            df_triaxial[f'{c}_Y (uT)'] = pd.Series([v[1] for v in df[c].values[0]])
            df_triaxial[f'{c}_Z (uT)'] = pd.Series([v[2] for v in df[c].values[0]])
          
        df_triaxial['Timestamp (ms)'] = recording['SU_INDIP']['LowerBack']['Timestamp']
        #---------FILTER RAW DATASET--------
        indoor = []
        stay_id = []
        for ctxs in os.listdir(os.path.join(data_folder, folder, patient,
                                            'Out of Lab/Contextual Factors')):
          if ctxs.startswith('stay'):
            staypts = pd.read_json(os.path.join(data_folder, folder, patient, 
                                  'Out of Lab/Contextual Factors', ctxs))['data']
          elif ctxs.startswith('per'):
            ctx = pd.read_json(os.path.join(data_folder, folder, patient,
                                  'Out of Lab/Contextual Factors', ctxs))['data'][0]
          elif ctxs.startswith('path'):
            path = pd.read_json(os.path.join(data_folder, folder, patient, 
                                  'Out of Lab/Contextual Factors', ctxs))
            
        filtered_ctx = filter_values(ctx, df_triaxial, ts)
        df_triaxial['Indoor Probability'] = pd.Series(checkIndoor(filtered_ctx, staypts))
                  
          
        df_triaxial['Patient ID'] = pd.Series([patient for _ in range(len(df_triaxial))])
        df_triaxial['Disease'] = pd.Series([folder for _ in range(len(df_triaxial))])
        print(len(df_triaxial))
        #df = pd.DataFrame(df)
        df_triaxial.set_index('Timestamp (ms)', inplace=True)

        if len(df_triaxial) > 0.5*ts:
          #full_df = full_df.append(df_triaxial)
          print('Total len: ', len(df_triaxial))
          print('Total null: ', df_triaxial[df_triaxial['Indoor Probability']==0.5]['Indoor Probability'].sum())
          print('Total indoor: ', df_triaxial[df_triaxial['Indoor Probability']==1]['Indoor Probability'].sum())
          print('Total null percentage: ', df_triaxial[df_triaxial['Indoor Probability']==0.5]['Indoor Probability'].sum()/len(df_triaxial)*100)
          print('Total indoor percentage: ', df_triaxial[df_triaxial['Indoor Probability']==1]['Indoor Probability'].sum()/len(df_triaxial)*100)
          print('Saving patient CSV file...')
          df_triaxial.to_csv(f'/content/drive/Shareddrives/Shared_Stefano_Vicenzo/Code/Data/valids/df_{patient}.csv')
        else:
          print('Too few samples for ID: ', patient)
        print(f'Total length of dataframe: {len(full_df)}')
        #print('Saving patient CSV file...')
        #df_triaxial.to_csv(f'/content/drive/Shareddrives/Shared_Stefano_Vicenzo/Code/Data/df_{patient}.csv')
        #print('Saved!')
        print('--------------------------')
      #except:
      #  print(f'Corrupted patient file: {patient}')
      #  continue
  return full_df

In [9]:
data_folder = '/content/drive/Shareddrives/Shared_Stefano_Vicenzo/Acquisitions/Data/TVS'
data_format = 'data.mat'

In [10]:
df = makeData(data_folder, data_format)

Processing patient id: 5003


<ipython-input-5-15018de51193>:18: DeprecationWarning: Please use `mat_struct` from the `scipy.io.matlab` namespace, the `scipy.io.matlab.mio5_params` namespace is deprecated.
  if isinstance(dict[key], spio.matlab.mio5_params.mat_struct):
<ipython-input-5-15018de51193>:29: DeprecationWarning: Please use `mat_struct` from the `scipy.io.matlab` namespace, the `scipy.io.matlab.mio5_params` namespace is deprecated.
  if isinstance(elem, spio.matlab.mio5_params.mat_struct):


1081922
Total len:  1081922
Total null:  0.0
Total indoor:  949800.0
Total null percentage:  0.0
Total indoor percentage:  87.7882139377885
Saving patient CSV file...
Total length of dataframe: 0
--------------------------
Processing patient id: 5008


<ipython-input-5-15018de51193>:18: DeprecationWarning: Please use `mat_struct` from the `scipy.io.matlab` namespace, the `scipy.io.matlab.mio5_params` namespace is deprecated.
  if isinstance(dict[key], spio.matlab.mio5_params.mat_struct):
<ipython-input-5-15018de51193>:29: DeprecationWarning: Please use `mat_struct` from the `scipy.io.matlab` namespace, the `scipy.io.matlab.mio5_params` namespace is deprecated.
  if isinstance(elem, spio.matlab.mio5_params.mat_struct):


905966
Total len:  905966
Total null:  33450.0
Total indoor:  626866.0
Total null percentage:  3.6921915391968354
Total indoor percentage:  69.19310437698545
Saving patient CSV file...
Total length of dataframe: 0
--------------------------
Processing patient id: 1006


...

<ipython-input-5-15018de51193>:18: DeprecationWarning: Please use `mat_struct` from the `scipy.io.matlab` namespace, the `scipy.io.matlab.mio5_params` namespace is deprecated.
  if isinstance(dict[key], spio.matlab.mio5_params.mat_struct):
<ipython-input-5-15018de51193>:29: DeprecationWarning: Please use `mat_struct` from the `scipy.io.matlab` namespace, the `scipy.io.matlab.mio5_params` namespace is deprecated.
  if isinstance(elem, spio.matlab.mio5_params.mat_struct):


946836
Total len:  946836
Total null:  115368.0
Total indoor:  716100.0
Total null percentage:  12.18458106789349
Total indoor percentage:  75.63083786421302
Saving patient CSV file...
Total length of dataframe: 0
--------------------------
Processing patient id: 4019


<ipython-input-5-15018de51193>:18: DeprecationWarning: Please use `mat_struct` from the `scipy.io.matlab` namespace, the `scipy.io.matlab.mio5_params` namespace is deprecated.
  if isinstance(dict[key], spio.matlab.mio5_params.mat_struct):
<ipython-input-5-15018de51193>:29: DeprecationWarning: Please use `mat_struct` from the `scipy.io.matlab` namespace, the `scipy.io.matlab.mio5_params` namespace is deprecated.
  if isinstance(elem, spio.matlab.mio5_params.mat_struct):


943138
Total len:  943138
Total null:  471569.0
Total indoor:  0.0
Total null percentage:  50.0
Total indoor percentage:  0.0
Saving patient CSV file...
Total length of dataframe: 0
--------------------------
Processing patient id: 3013


FileNotFoundError: ignored

In [ ]:
df.head()

""


Index([], dtype='object')

In [ ]:
#df.to_csv('/content/drive/MyDrive/Tesi/test_df_5003.csv')